In [2]:
import geopandas as gpd
from matplotlib import pyplot as plt

In [3]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
tiles_df = gpd.read_file('output/covering_tiles.kml', driver='KML')
print(repr(tiles_df.crs))
print(tiles_df)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

     Name Description                                           geometry
0                      POLYGON Z ((-120.28118 48.72093 0.00000, -118....
1                      POLYGON Z ((-119.67207 47.82260 0.00000, -118....
2                      POLYGON Z ((-119.71936 48.72091 0.00000, -118....
3                      POLYGON Z ((-113.43120 42.42690 0.00000, -112....
4                      POLYGON Z ((-112.21648 42.44624 0.00000, -110....
...   ...         ...                                                ...
1179                   POLYGON Z ((29.99818 -26.18805 0.00000, 31.096...
1180                   POLYGON Z ((136.08714 -34.33630 0.00000, 137.2...
1181                   POLYGON Z 

In [187]:
from shapely.geometry.multilinestring import MultiLineString
from shapely.geometry import MultiPolygon, Polygon
from shapely.geometry.linestring import LineString
from shapely.ops import split
import shapely.ops as shops
import shapely.geometry as shgeom
import shapely


"""
Can't be assed to axis-align the box, so just do two separate lerps.

Given this square:
1 --> 2
^     |
|     v
0 <-- 3

left -> right: lerp 0, 1 -> 3, 2
top -> down: lerp 1, 2 -> 0, 3
"""

def plot_geom(geom, **kwargs):
    gpd.GeoSeries(geom).plot(ax=ax, **kwargs)

# first lerp: 0, 3 -> 1, 2
# https://stackoverflow.com/a/58284215
def split_1d(geom, n, direction, debug=False):
    # coords = list(geom.exterior.coords)
    coords = list(shgeom.Point(c) for c in geom.exterior.coords)
    if len(coords) == 5:
        for i, c in enumerate(coords):
            if i == len(coords)-1:
                d = coords[0]
            else:
                d = coords[i+1]
            if c == d:
                if debug: print('Remove', i)
                del coords[i]
                break
    assert len(coords) == 4

    def offset_from_centroid(c):
        # return (c[0] - geom.centroid.x, c[1] - geom.centroid.y)
        return shapely.affinity.translate(c, -geom.centroid.x, -geom.centroid.y)
    min_coord_i = 0
    for i, c in enumerate(coords):
        off = offset_from_centroid(c)
        # if off[0] < 0 and off[1] < 0:
        if off.x < 0 and off.y < 0:
            min_coord_i = i
            if debug: print('Rotate', i, 'to first index')
    coords = coords[min_coord_i:] + coords[:min_coord_i]

    # Assume the following:
    # 1. 4 coordinates are the 4 bounds of the rectangle
    # 2. Coordinates are in a linear ring
    # 3. Bottom left coordinate is at coords[0]

    if debug:
        for i, c in enumerate(coords):
            plot_geom(shgeom.Point(c), color=(0, 0, 1, (i+1)/len(coords)))

    splitters = []
    if direction == 'left_right':
        ls_a = LineString((coords[0], coords[3]))
        ls_b = LineString((coords[1], coords[2]))
    elif direction == 'top_down':
        ls_a = LineString((coords[1], coords[0]))
        ls_b = LineString((coords[2], coords[3]))
    else:
        raise NotImplementedError(direction)
    # Test that top and bot are correctly named
    if debug:
        plot_geom(ls_a, color='red')
        plot_geom(ls_b, color='green')
    offset = 0.2
    for i in range(1, n):
        percent = i / n
        interp_a_ls = ls_a.interpolate(percent, normalized=True)
        interp_b_ls = ls_b.interpolate(percent, normalized=True)
        new_interp_a_ls = shgeom.Point(
            (interp_a_ls.x - ((interp_b_ls.x - interp_a_ls.x) * offset)),
            (interp_a_ls.y - ((interp_b_ls.y - interp_a_ls.y) * offset))
            )
        new_interp_b_ls = shgeom.Point(
            (interp_b_ls.x - ((interp_a_ls.x - interp_b_ls.x) * offset)),
            (interp_b_ls.y - ((interp_a_ls.y - interp_b_ls.y) * offset))
            )
        if debug:
            plot_geom(new_interp_a_ls, color=(1, 0, 0, percent))
            plot_geom(new_interp_b_ls, color=(0, 1, 0, percent))
        split_ls = LineString((new_interp_b_ls, new_interp_a_ls))
        splitters.append(split_ls)

    splits = []
    q = geom
    for i, splitter in enumerate(splitters):
        if debug:
            plot_geom(q, color=(0, 0, 0, 0), edgecolor=(0, 0, 1), hatch='o')
            plot_geom(splitter, color=(0, 0, 1))
        split_result = split(q, splitter)
        if len(split_result) == 2:
            p, q = split_result
            splits.append(p)
        else:
            if splitter.intersects(q):
                print('line intersects but does not split')
            else:
                print('line does not intersect')
            raise Exception(f'iteration {i} {len(split_result)=}')
    splits.append(q)
    return splits

import time

times = []
for tile_geom in tiles_df.geometry[:5]:
    begin = time.time()
    small_tiles = []

    n = 10
    splits = split_1d(tile_geom, n, 'left_right')
    for i, s in enumerate(splits):
        s_split = split_1d(s, n, 'top_down')
        small_tiles += s_split
    end = time.time()
    times.append(end-begin)
    
    # ax = plt.gca()
    # for i in range(n):
    #     for j in range(n):
    #         plot_geom(small_tiles[i*n+j], color=(1, i/n, j/n))
    # plt.show()
print('Average time:', sum(times)/len(times))

Average time: 0.0705474853515625
